<a href="https://colab.research.google.com/github/aafaf-arharas/ML_TPs/blob/main/02_PMC_2couches_etud.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Perceptron multicouches et Keras

Définition d'un perceptron multicouches à deux couches cachées pour la classification de données MNIST


## MNIST 

La base de données MNIST (Mixed National Institute of Standards and Technology), est une base de données de chiffres manuscrits. C’est une base de données standard pour le test de nouveaux algorithmes de reconnaissance de ces chiffres. Elle est composée de 60000 images d’apprentissage et 10000 images de test. Les images en noir et blanc, normalisées centrées de 28 pixels de côté.

![mnist.png](./mnist.png)




# Import

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow.keras as tk
from tensorflow.keras import Sequential


from tensorflow.keras.layers import Input, Dense, Activation,Lambda
from tensorflow.keras.models import Model

from tensorflow.keras.optimizers import Adam


# Données

On charge ensuite les données MNIST. Les paramètres de la base sont récupérés (nombre d'exemples, de classes, taille de la rétine).

In [2]:
(x_train, y_train), (x_test, y_test) =tk.datasets.mnist.load_data()

num_examples = x_train.shape[0] 
num_test = x_test.shape[0]
num_input = x_train.shape[1]*x_train.shape[2]
num_classes = 10

img_size = x_train.shape[1] 
img_shape = (img_size, img_size)

x_train = x_train.reshape((num_examples, num_input))/255
x_test  = x_test.reshape((num_test, num_input))/255

# normalisation



print('Taille de la rétine : ',num_input)
print("Nombre d'exemples : ",num_examples)

y_train = tk.utils.to_categorical(y_train, num_classes)
y_test = tk.utils.to_categorical(y_test, num_classes)


11501568/11490434 [==============================] - 0s 0us/step
Taille de la rétine :  784
Nombre d'exemples :  60000


# Réseau

## Paramètres du réseau

Dans un premier temps, on définit les paramètres du réseau : 
- 256 neurones cachés dans chaque couche cachée, 
- un apprentissage par batchs de taille 100
- 15 itérations pour l'apprentissage


In [3]:

lr = 0.001
num_epochs = 15
batch_size = 100

# Nombre de neurones sur les deux couches cachées
num_hidden_1 = 256 
num_hidden_2 = 256 

## Définition du réseau

On construit alors le modèle :
- le réseau
- la fonction de coût à optimiser : dans le cas d'un problème de classification, la fonction d'entropie croisée calculée entre la sortie théorique et la sortie calculée par le modèle est adéquate
- la méthode d'optimisation utilisée (descente de gradient) : ici, l'algorithme [ADAM](https://arxiv.org/abs/1412.6980) est utilisé

In [11]:
#TODO : réseau
#Définition des entrees
entree = Input(shape=(784), name="input_layer")
x = Dense(units=num_hidden_1, name="dense_layer_1")(entree)
x = Activation('relu', name="activation_layer")(x)

def custom_layer(tensor):
    return tensor + 2

out = Dense(units=10, name="dense_layer_2")(x)
#x = Lambda (custom_layer, name="lambda_layer")(x)



# Modele
model = Model(entree , out)
model.summary()





Model: "model_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_layer (InputLayer)     [(None, 784)]             0         
_________________________________________________________________
dense_layer_1 (Dense)        (None, 256)               200960    
_________________________________________________________________
activation_layer (Activation (None, 256)               0         
_________________________________________________________________
dense_layer_2 (Dense)        (None, 10)                2570      
Total params: 203,530
Trainable params: 203,530
Non-trainable params: 0
_________________________________________________________________


In [13]:
#TODO : optimiseur et fonction de perte
model.compile(loss="binary_crossentropy",optimizer="Adam",metrics=['acc'])

On entraîne le modèle

In [14]:
#TODO ;: entraînement
#Entrainement
model.fit(x_train,y_train, batch_size =100, epochs =15)


Epoch 1/15
600/600 [==============================] - 3s 5ms/step - loss: 0.4942 - acc: 0.6988
Epoch 2/15
600/600 [==============================] - 3s 4ms/step - loss: 0.5647 - acc: 0.6734
Epoch 3/15
600/600 [==============================] - 3s 5ms/step - loss: 0.7823 - acc: 0.5250
Epoch 4/15
600/600 [==============================] - 3s 4ms/step - loss: 0.8393 - acc: 0.4967
Epoch 5/15
600/600 [==============================] - 3s 4ms/step - loss: 0.8265 - acc: 0.5679
Epoch 6/15
600/600 [==============================] - 3s 5ms/step - loss: 0.9323 - acc: 0.4971
Epoch 7/15
600/600 [==============================] - 3s 5ms/step - loss: 1.0068 - acc: 0.4833
Epoch 8/15
600/600 [==============================] - 3s 4ms/step - loss: 1.0531 - acc: 0.4101
Epoch 9/15
600/600 [==============================] - 3s 4ms/step - loss: 1.1188 - acc: 0.4670
Epoch 10/15
600/600 [==============================] - 3s 4ms/step - loss: 0.9753 - acc: 0.5303
Epoch 11/15
600/600 [============================

In [15]:
print("Précision ={0:5.3f} ".format(hist.history.get('acc')[-1]))

NameError: ignored

Puis on l'évalue sur l'ensemble de test 

In [16]:
score = model.evaluate(x_test,y_test)

x = list(range(1,num_epochs+1))
l = hist.history['loss']
plt.subplots(1,1)
plt.xlabel('epoch')
plt.ylabel('Entropie croisée')
plt.title("Test : Score = {0:5.3f}, Précision = {1:5.3f}".format(score[0], score[1]))
plt.plot(x,l)
plt.show()
plt.tight_layout()

predicted_classes = np.argmax(model.predict(x_test), axis=-1)
y = np.argmax(y_test,axis=1)

incorrects = np.nonzero(predicted_classes != y)[0]
plt.figure(figsize=(10,10))
for i in range(0,9):
    plt.subplot(3,3,i+1)
    plt.imshow(x_test[incorrects[i]].reshape(28,28), cmap='gray', interpolation='none')
    plt.title( "Prédit/vrai : {}/{}".format(predicted_classes[incorrects[i]], y[incorrects[i]]))
    plt.xticks([])
    plt.yticks([])
plt.tight_layout()
print("Nombre d'erreurs {}/{}\n\n".format(incorrects.size,y.size))


313/313 [==============================] - 1s 2ms/step - loss: 0.7334 - acc: 0.6517


NameError: ignored